# Filtering

## Camera filtering:

needed inputs: wheel_speed, cameras, camera state, dt
fixed variables: wheelbase (measured), GPS noise on axes (measured)
outputs: 

In [ ]:
import numpy as np



class KalmanFilter:
    def __init__(self, x0):
        self.ds = 0.0
        self.dtheta = 0.0
        self.x = np.zeros((3, 1))
        self.dt = 0.0
        self.wheelbase = 0.0
        self.sigma_gpsx = 0.0
        self.sigma_gpsy = 0.0
        self.sigma_gpstheta = 0.0
        self.A = np.array([
            [1, 0, -self.ds * np.sin(self.x[2]+self.dtheta/2)],
            [0, 1, self.ds * np.cos(self.x[2]+self.dtheta/2)],
            [0, 0, 1],
        ])
        self.B = np.array([
            [0.5 * self.dt * np.cos(self.x[2]), 0.5 * self.dt * np.cos(self.x[2])],
            [0.5 * self.dt * np.sin(self.x[2]), 0.5 * self.dt * np.sin(self.x[2])],
            [1.0 / self.wheelbase, -1.0 / self.wheelbase],
        ])
        self.H = np.array([
            [1, 0, 0],  # GPS x
            [0, 1, 0],  # GPS y
            [0, 0, 1],  # GPS theta
        ])
        self.Q = np.diag([0.1, 0.1, 0.1])
        self.R = np.diag([self.sigma_gpsx**2, self.sigma_gpsy**2, self.sigma_gpstheta**2])
        self.P = np.diag([1, 1, 1])
        self.x = x0
    def robot_speed(speed_left, speed_right):
        return (speed_left + speed_right) / 2.0

    def update_kalman(self, wheel_speed_left, wheel_speed_right, camera_state, dt, camera_data):
        self.dt = dt
        v = self.robot_speed(wheel_speed_left, wheel_speed_right)  # Robot speed
        self.dtheta = self.dt*(wheel_speed_right - wheel_speed_left) / self.wheelbase  # Robot angular speed

        self.ds = v * self.dt  # Robot displacement
        # State vector: [x, y, theta]
        u = [wheel_speed_left, wheel_speed_right]  # Replace with actual wheel velocity inputs
        self.x = self.A.dot(self.x) + self.B.dot(u)

        if(camera_state == on):
            # Prediction covariance
            self.P = self.A.dot(self.P).dot(self.A.T) + self.Q

            # Update
            self.K = self.P.dot(self.H.T).dot(np.linalg.inv(self.H.dot(self.P).dot(self.H.T) + self.R))
            self.z = camera_data  # Replace with actual sensor measurements (X GPS, Y GPS, Theta GPS)
            self.x = self.x + self.K.dot(self.z - self.H.dot(self.x))

            # Update covariance
            self.P = (np.eye(3) - self.K.dot(self.H)).dot(self.P)

        # Extract final estimates
        estimated_position = self.x[:2]
        estimated_orientation = self.x[2]
        return estimated_position, estimated_orientation
        


In [1]:
from filterpy.kalman import KalmanFilter

In [ ]:
f = KalmanFilter(dim_x=3, dim_z=3)
f.x = np.zeros((3, 1))
vx = robot_speed(get_wheel_speed())
ds = dt*vx
dtheta = dt*((get_wheel_speed()[0] - get_wheel_speed()[1])/wheelbase)
f.F = np.array([
    [1, 0, -ds * np.sin(f.x[2]+dtheta/2)],
    [0, 1, ds * np.cos(f.x[2]+dtheta/2)],
    [0, 0, 1],
])

